[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/atmos-cloud-sim-uj/PySDM-examples.git/main?urlpath=lab/tree/PySDM_examples/Kreidenweis_et_al_2003/demo.ipynb)    
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/atmos-cloud-sim-uj/PySDM-examples/blob/main/PySDM_examples/Kreidenweis_et_al_2003/demo.ipynb)

#### based on Jaruga and Pawlowska (Geosci. Model Dev. 11)  "_libcloudph++ 2.0: aqueous-phase chemistry extension of the particle-based cloud microphysics scheme_"   
https://doi.org/10.5194/gmd-11-3623-2018

In [1]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install git+https://github.com/atmos-cloud-sim-uj/PySDM-examples.git
    !ldconfig

In [2]:
from PySDM_examples.Kreidenweis_et_al_2003 import Settings, Simulation
from PySDM import products as PySDM_products
from PySDM_examples.utils.show_plot import show_plot
from PySDM_examples.utils.widgets import display, FloatProgress
from PySDM.physics import si
from PySDM.physics.constants import ppt, ppb
import numpy as np
import os
from matplotlib import pyplot

/home/slayoo/devel/PySDM/PySDM/backends/__init__.py:29: UserWarning: CUDA library found but cuInit() failed (error code: 999; message: unknown error)
  warnings.warn(


In [3]:
n_points = 12 if 'CI' not in os.environ else 2
nsd = np.logspace(0, n_points-1, num=n_points, base=2.0, dtype=int)

In [4]:
default_settings = Settings(1,1,1)
products = (
    PySDM_products.PeakSupersaturation(),
    PySDM_products.CloudDropletConcentration(radius_range=default_settings.cloud_radius_range),
    PySDM_products.pH(radius_range=default_settings.cloud_radius_range),
    PySDM_products.AqueousMoleFraction('S_VI')
)

In [ ]:
simulations = []
progress = FloatProgress(value=0.0, min=0.0, max=1.0)
display(progress)
for it in range(n_points):
    settings = Settings(
        dt=1*si.s, 
        n_sd=nsd[it], 
        n_substep=10 if 'CI' not in os.environ else 2
    )
    settings.output_interval = 25 * si.s
    simulation = Simulation(settings, products)
    output = simulation.run()
    simulations.append({'settings': settings, 'output': output})
    progress.value += 1/n_points

FloatProgress(value=0.0, max=1.0)

In [ ]:
x = np.log2(nsd)
smax = []
droplet_number = []
pH = []
sulfate_ppt = []
for simulation in simulations:
    smax.append(np.nanmax(simulation['output']["S_max"]))
    droplet_number.append(np.nanmax(simulation['output']["n_c_cm3"]))
    pH.append(simulation['output']["pH"][-1])
    S_VI = simulation['output']["aq_S_VI_ppb"]
    sulfate_ppt.append((S_VI[-1] - S_VI[0]) * ppb / ppt)

In [ ]:
fig, axs = pyplot.subplots(2, 2, figsize=(10,7))

axs[0][0].plot(x, droplet_number)
axs[0][0].set_ylabel('Droplet conc. [cm-3]')
axs[0][0].set_ylim(0, 1800)

axs[0][1].plot(x, smax)
axs[0][1].set_ylabel('Max. supersat [%]')
axs[0][1].set_ylim(0.24, 0.61)

axs[1][0].plot(x, pH)
axs[1][0].set_ylabel('Average pH')
axs[1][0].set_ylim(4.7, 5.2)

axs[1][1].plot(x, sulfate_ppt)
axs[1][1].set_ylabel('Total sulfate production [ppt]')
axs[1][1].set_ylim(0*135, 180)  # TEMP 0* !!!

for axss in axs:
    for ax in axss:
        ax.set_xlabel('ln_2(number of super-droplets)')
        ax.grid()
        ax.set_xlim(0,10)

show_plot()